# Deutsch algorithm with ProjectQ 

This tutorial will go through the following contents: 

* Fundamental gate operations
    * NOT
    * Hadamard 
    * CNOT


* Two qubit Deutsch algorithm


In [1]:
from __future__ import print_function
#Importing projectQ gates
from projectq.ops import H, NOT, CNOT, Measure
from projectq import MainEngine
eng = MainEngine()
def print_wavefunction(qureg, nqubits, mask, opt='not'):
  # print wavefunction
    eng.flush()
    print("   |Psi> = ", end='')
    i=0
    for s in range(2 ** nqubits):
        bit_string = format(s, mask)
        result=eng.backend.get_amplitude(bit_string, qureg)
        if (opt.find('all')!=-1):
            print("%.1f|%s> " % (result.real,bit_string), end='')
            if (s<2**nqubits-1):
                print("+", end='')
        else:       
            if (abs(result.real)!=0.0):
                if i != 0 and result.real >0: print("+ ", end='')
                print("%.2f|%s> " % (result.real,bit_string), end='')
                i=i+1
    print()
    return

def print_qubits(qureg):
    eng.flush()
    i=0
    try:
        for q in qureg:
            print("   Measured Q{}: |{}>".format(i, int(q)))### multiple qubit case
            i=i+1
    except:
        print("   Measured: |{}>".format(int(qureg))) ### single qubit
    return 

def measure_qubits(qureg):
    Measure | qureg
    eng.flush()
    i=0
    try:
        for q in qureg:
            print("   Measured Q{}: |{}>".format(i, int(q)))### multiple qubit case
            i=i+1
    except:
        print("   Measured: |{}>".format(int(qureg))) ### single qubit
    return 

(Note: This is the (slow) Python simulator.)


## Fundamental gate operations
### Initializing a two qubit state
We initialize a two qubit state: 
   
   $|\Psi\rangle = |00\rangle$

both are at |0> state.

In [2]:
# initialize variables
eng = MainEngine() ## engine
nqubits=2
mask='02b'
qureg = eng.allocate_qureg(nqubits) ## allocate 2 qubit, stored at qureg
print("\n*Allocate %s qubits" %nqubits)
print_wavefunction(qureg, nqubits, mask, 'all')

(Note: This is the (slow) Python simulator.)

*Allocate 2 qubits
   |Psi> = 1.0|00> +0.0|01> +0.0|10> +0.0|11> 


## NOT gate

* NOT gate transformation

    $|0\rangle \longrightarrow |1\rangle$

    $|1\rangle \longrightarrow |0\rangle$


* Matrix representation is 

    NOT = $\left(
    \begin{matrix}
    0 & 1 \\
    1 & 0
    \end{matrix}\right)
    $

In [3]:
print("*Initial state")
print_wavefunction(qureg, nqubits, mask)
print("\n*Apply NOT gate to the first qubit")
NOT | qureg[0]

print_wavefunction(qureg, nqubits, mask)

print("\n*Apply NOT gate to the first qubit again")
NOT | qureg[0]
print_wavefunction(qureg, nqubits, mask)

*Initial state
   |Psi> = 1.00|00> 

*Apply NOT gate to the first qubit
   |Psi> = 1.00|10> 

*Apply NOT gate to the first qubit again
   |Psi> = 1.00|00> 


## Hadamard (H) gate

* Hadamard transformation

    $|0\rangle \longrightarrow \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$ 

    $|1\rangle \longrightarrow \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$ 

* The matrix representation is: 

    H = $
\frac{1}{\sqrt{2}}\left(
\begin{matrix}
1 & 1 \\
1 & -1
\end{matrix}\right)
$

* Two Hadamard gate together equals to identify

    $H\times H=I$


In [4]:
print("*Initial state")
print_wavefunction(qureg, nqubits, mask)
print("\n*Apply Hadamard gate")
H | qureg[0]
print_wavefunction(qureg, nqubits, mask)
print("\n*Apply Hadamard gate again")
H | qureg[0]
print_wavefunction(qureg, nqubits, mask)

*Initial state
   |Psi> = 1.00|00> 

*Apply Hadamard gate
   |Psi> = 0.71|00> + 0.71|10> 

*Apply Hadamard gate again
   |Psi> = 1.00|00> 


## CNOT

Control NOT: apply NOT to a qubit if the control qubit is at state |1>: 


$
 \quad |00\rangle \longrightarrow|00\rangle\\
\quad |01\rangle  \longrightarrow |01\rangle\\
\quad |10\rangle  \longrightarrow|11\rangle\\
\quad |11\rangle  \longrightarrow |10\rangle
$

Matrix representation of CNOT operator is: 

CNOT = $
\left(
\begin{matrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 
\end{matrix}\right)
$


In [5]:
print("Initial state: ")
print_wavefunction(qureg, nqubits, mask)
print("\n*CNOT q0->q1")
CNOT | (qureg[0], qureg[1])  ## operate on the second qubit based on the state of the first qubit
print_wavefunction(qureg, nqubits, mask)
NOT | qureg[0]
print("\n*Apply NOT gate to q0 ")
print_wavefunction(qureg, nqubits, mask)
print("\n*CNOT q0->q1")
CNOT | (qureg[0], qureg[1])  ## operate on the second qubit based on the state of the first qubit
print_wavefunction(qureg, nqubits, mask)
print("\n*CNOT q0->q1")
CNOT | (qureg[0], qureg[1])  ## operate on the second qubit based on the state of the first qubit
print_wavefunction(qureg, nqubits, mask)

print("\n*Apply NOT to q0 to go back to initial state: ")
NOT | qureg[0]
print_wavefunction(qureg, nqubits, mask)

Initial state: 
   |Psi> = 1.00|00> 

*CNOT q0->q1
   |Psi> = 1.00|00> 

*Apply NOT gate to q0 
   |Psi> = 1.00|10> 

*CNOT q0->q1
   |Psi> = 1.00|11> 

*CNOT q0->q1
   |Psi> = 1.00|10> 

*Apply NOT to q0 to go back to initial state: 
   |Psi> = 1.00|00> 


## Deutsch algorithm

In the Deutsch-Jozsa problem, we are given a black box quantum computer known as an oracle that implements some function $f:\{0,1\}^{n}\rightarrow \{0,1\}$. In other words, it takes n-digit binary values as input and produces either a 0 or a 1 as output for each such value. We are promised that the function is either constant (0 on all outputs or 1 on all outputs) or balanced[3] (returns 1 for half of the input domain and 0 for the other half); the task then is to determine if f is constant or balanced by using the oracle.

$$ U_f |x\rangle |y\rangle = |x\rangle|f(x)\oplus y\rangle $$

<img src="dja.svg">
Deutsch-Jozsa algorithm


Deutsch's algorithm is a special case of the general Deutsch–Jozsa algorithm. We need to check the condition $f(0)=f(1)$. It is equivalent to check $f(0)\oplus f(1)$ (where  $\oplus$  is addition modulo 2, which can also be viewed as a quantum XOR gate implemented as a Controlled NOT gate), if zero, then $f$ is constant, otherwise $f$ is not constant.

We begin with the two-qubit state $|0\rangle |1\rangle$  and apply a Hadamard transform to each qubit. This yields

$${\frac  {1}{2}}(|0\rangle +|1\rangle )(|0\rangle -|1\rangle ).$$


We are given a quantum implementation of the function $f$ that maps $|x\rangle |y\rangle$  to $|x\rangle |f(x)\oplus y\rangle$. Applying this function to our current state we obtain


$${\frac{1}{2}}\left(|0\rangle (|f(0)\oplus 0\rangle -|f(0)\oplus 1\rangle \right)+|1\rangle \left(|f(1)\oplus 0\rangle -|f(1)\oplus 1\rangle \right)\\
=\frac{1}{2}((-1)^{f(0)}|0\rangle (|0\rangle - |1\rangle + (-1)^{-1}|1\rangle (|0\rangle - |1\rangle))\\
=(-1)^{f(0)}\frac{1}{2}\left(|0\rangle + (-1)^{f(0)\oplus f(1)}|1\rangle\right)(|0\rangle - |1\rangle)
.$$

We ignore the last bit and the global phase and therefore have the state

$$\frac {1}{\sqrt {2}}(|0\rangle +(-1)^{f(0)\oplus f(1)}|1\rangle ).$$


Applying a Hadamard transform to this state we have

$$\frac  {1}{2}(|0\rangle +|1\rangle +(-1)^{{f(0)\oplus f(1)}}|0\rangle -(-1)^{{f(0)\oplus f(1)}}|1\rangle )\\ ={\frac  {1}{2}}((1+(-1)^{{f(0)\oplus f(1)}})|0\rangle +(1-(-1)^{{f(0)\oplus f(1)}})|1\rangle ).$$

We see that if $f(0)=f(1)$, we get the final state to be
$$\frac{1}{\sqrt{2}}|0\rangle(|0\rangle -|1\rangle )$$
Otherwise, we get
$$\frac{1}{\sqrt{2}}|1\rangle(|0\rangle -|1\rangle )$$

We could also apply Hadamard gate to the second qubit, so that 
We see that if $f(0)=f(1)$, we get the final state to be

$$f(0)=f(1), \quad |\Psi\rangle = |00\rangle\quad \quad\\
f(0)\ne f(1), \quad |\Psi\rangle = |01\rangle $$


In [6]:
# initialize variables
nqubits=2
mask='02b'

# create a main compiler engine
eng = MainEngine()

# allocate qubits
qureg = eng.allocate_qureg(nqubits)
NOT | qureg[1]

def Constant_Oracle(qureg, f=0):
    print("    Constant oracle: f(0)=f(1)=%s"%f)
    if (f==1):
        NOT | qureg[1]
        
def Balanced_Oracle(qureg, f=[0, 1]):
    print("    Balanced oracle: f(0)=%s, f(1)=%s"%(f[0], f[1]))
    if f[0]==0:
        CNOT | (qureg[0], qureg[1])
    else:
        CNOT | (qureg[0], qureg[1])
        NOT | qureg[1]

print("\n*Inital state |01>: ")
print_wavefunction(qureg, nqubits, mask)
print("\n*Apply Hadamard to each qubit: ")
# apply gates
H | qureg[0]
H | qureg[1]
print_wavefunction(qureg, nqubits, mask)

print("\n*Apply Oracle: ")

Balanced_Oracle(qureg, f=[0, 1])

print_wavefunction(qureg, nqubits, mask)

print("\n*Apply Hadamard to Q0")
H | qureg[0]
H | qureg[1]
print_wavefunction(qureg, nqubits, mask)

# measure qubits

print("\n*Measure the first")
measure_qubits(qureg[0])


(Note: This is the (slow) Python simulator.)

*Inital state |01>: 
   |Psi> = 1.00|01> 

*Apply Hadamard to each qubit: 
   |Psi> = 0.50|00> -0.50|01> + 0.50|10> -0.50|11> 

*Apply Oracle: 
    Balanced oracle: f(0)=0, f(1)=1
   |Psi> = 0.50|00> -0.50|01> -0.50|10> + 0.50|11> 

*Apply Hadamard to Q0
   |Psi> = 1.00|11> 

*Measure the first
   Measured: |1>


# Exercises

* Implementing a three qubit Deutsch's algorithm
* Implementing a four qubit Quantum Fourier Transformation according to the following quantum circuit.